# Exploring data

In [1]:
import pandas as pd
import numpy as np 
import warnings; warnings.simplefilter('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from ast import literal_eval
from matplotlib import pyplot as plt
from nltk.stem.snowball import SnowballStemmer

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [2]:
#reading the dataset 
# genome_score = pd.read_csv("ml-20m/genome-scores.csv")
# genome_tag = pd.read_csv("ml-20m/genome-tags.csv")
# link = pd.read_csv("ml-20m/links.csv")
# movie =  pd.read_csv("ml-20m/movies.csv")
# rating = pd.read_csv("ml-20m/ratings.csv")
# tags = pd.read_csv("ml-20m/tags.csv")

In [3]:
# genome_score.head(10)

In [4]:
# genome_tag.head(10)

In [5]:
# link.head(10)

In [6]:
# movie.head(10)

In [7]:
# rating = rating.head(10 )

In [8]:
# tags.head(10)

## Simple Recommender
### Sort movie by its popularity
How to calculate movie popularity? 

According to imdb, movie rating is calculated as follow:

Weight Rated = (v/(m+v)*R) + (m/(v+m)*C)

For v = numbers of votes for the movie

For m = The minimum votes required to be on the chart

For R = The average rating of the movie 

For C = The mean vote across the whole record

### Request data from database




In [9]:
import requests
import pandas as pd

url = "https://api.themoviedb.org/3/movie/{}"
latest_movieID = 528128
first_movieID = 1
D = {'api_key':'7cfc8557ee5d234bff85e81eb103c210'}
api = requests.get(url.format(latest_movieID) , params = D)
movie_json = api.json()
list(movie_json.keys())


['adult',
 'backdrop_path',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [10]:
# import time
# #query every movive for the movie database 

# for index in range (4138, 13203):
#     movie_list= []
#     for movieID in range(0,40):
#         api = requests.get(url.format(movieID + index*40),params = D)
#         print(movieID + index*100)
#         if(api.status_code == 404):
#             print("ERROR 404: MOVIE NOT FOUND")
#         elif(api.status_code == 200):
#             print("SUCCESS")
#             movie = api.json()
#             movie_list.append(movie)
#         else:
#             print(api.status_code)
#     if(len(movie_list) != 0):
#         movie_data = pd.DataFrame(movie_list)
#         average_rate = movie_data.vote_average.mean()
#         movie_data['average_rate'] = average_rate
#         csv_file = "movie_data{}.csv"
#         movie_data.to_csv(csv_file.format(index))
#     time.sleep(10)

In [11]:
# import warnings; warnings.simplefilter('ignore')

movie_data = pd.read_csv("md.csv")
linked_small = pd.read_csv("links_small.csv")
linked_small = linked_small[['movieId','imdbId', 'tmdbId']]
linked_small

,movieId,imdbId,tmdbId
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862
5,6,113277,949
6,7,114319,11860
7,8,112302,45325
8,9,114576,9091
9,10,113189,710


In [12]:

m = movie_data['vote_count'].quantile(0.95)
C = movie_data['vote_average'].mean()

def weightRate(movie):
    #v : number of vote 
    v = movie['vote_count']
    
    #m : minimum vote to be in the list 
    #R : average rating of a movie 
    R = movie['vote_average']
    #C: mean vote accross every record
    return R*v/(v+m) + C*m/(v+m)
    

movie_data['weight_rate'] = movie_data.apply(weightRate, axis = 1)
mdRanked = movie_data.sort_values(by= ['weight_rate'], ascending = False)

## Content based recommender
### What's the different 
This recommender system based on the used preference, more details of its, look at the google docs sheet 


In [13]:
# genome_score.head(10)

In [14]:
# genome_tag.head(10)

In [15]:
# link.head(10)

In [16]:
# movie.head(10)

In [17]:
# rating.head(10)

In [18]:
# tags.head(10)

In [19]:
linked_small = linked_small.astype(int)

# for link in linked_small['tmdbId']:
#     print(link)

link_small_tmdbId = linked_small[linked_small['tmdbId'].notnull()]['tmdbId'].astype(int)
movie_data = movie_data[movie_data['id'].notnull()]
movie_data['id']= movie_data['id'].astype(int)
# print(movie_data['imdb_id'])
# for ID in movie_data['id']:
#     if(ID.find("-") != -1):
#         movie_data = movie_data[movie_data.id != ID]
        
# # print(link_small_imdbId[link_small_imdbId =='155'])
# # print(link_small_tmdbId)
# for i in link_small_tmdbId:
#     if (i == 155):
#         print(i)

In [20]:
movie_data.to_csv('movie_data.csv')
# print(link_small_imdbId.head(10))
# print(movie_data[movie_data.title == "The Dark Knight"]) 

tmdb_md = movie_data[movie_data['id'].isin(link_small_tmdbId)]
# print(Imdb_md)


tmdb_md['tagline'] = tmdb_md['tagline'].fillna('')
tmdb_md['overview'] = tmdb_md['overview'].fillna('')
tmdb_md['description'] = tmdb_md['tagline'] + tmdb_md['overview']

# print(Imdb_md[Imdb_md.title == 'The Dark Knight'])

df = tmdb_md[tmdb_md.title == "The Dark Knight"]
tmdb_md.shape

(7082, 12)

In [51]:
# small_description = tmdb_md['description']
small_description = pd.Series()
for movie in tmdb_md:
    small_description.append(movie['description'], ignore_index = True)
    
print(type(small_description))
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
# tfidf_matrix = tf.fit_transform(Imdb_md['description'])
tfidf_matrix = tf.fit_transform(small_description)


# for i in tf.get_feature_names():
#     print(i)
# print(len(tf.get_feature_names()))
# print(type(tfidf_matrix))
# print(tfidf_matrix.toarray())



TypeError: string indices must be integers

In [22]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# print(list(enumerate(cosine_sim[6900])))
# print(cosine_sim.shape)
# print(cosine_sim)
# print(list(enumerate(cosine_sim[0])))
# for item in list(enumerate(cosine_sim[0])):
#     print(item)

In [23]:
tmdb_md = tmdb_md.reset_index()

In [45]:
titles = tmdb_md['title']
# print(titles)
# df = tmdb_md[tmdb_md.title == 'The Dark Knight']
# print(df)
    
indices = pd.Series(data = tmdb_md.index, index =titles )
# print(indices["The Confessional"])
# print(indices)

# print(sim_score)
def getRecommendation(title):
    title_index = indices[title]
    print(title_index)
    sim_score = list(enumerate(cosine_sim[title_index]))
#     print(sim_score)
    sim_score = sorted(sim_score, key = lambda x: x[1], reverse = True)
    sim_score = sim_score[1:31]
    movie_indice = [i[0] for i in sim_score]
    
    return indices.iloc[movie_indice] 

In [46]:
getRecommendation('The Dark Knight')    

2


title
The Dark Knight Rises                        89
Batman Forever                             6975
Batman Returns                              986
Batman: Under the Red Hood                  611
Batman: The Dark Knight Returns, Part 2     509
Batman                                      488
Batman: Year One                           1474
Batman: Mask of the Phantasm               1368
JFK                                         609
Batman: The Dark Knight Returns, Part 1     620
Sherlock Holmes: A Game of Shadows          403
Batman Begins                               126
Q & A                                      4620
Law Abiding Citizen                         412
The File on Thelma Jordon                  5336
Flying Tigers                              5451
Night Falls on Manhattan                   4563
The Young Savages                          5702
Batman v Superman: Dawn of Justice         4147
Batman & Robin                             7080
Rush Hour 3                       

In [26]:
credits = pd.read_csv("credits.csv")
keywords = pd.read_csv("keywords.csv")


credits['id'] = credits['id'].astype(int)
keywords['id'] = keywords['id'].astype(int)
movie_data['id'] = movie_data['id'].astype(int)
# movie_data
# credits 
# keywords

In [27]:
movie_data =pd.merge(movie_data, credits, how = "inner", on= 'id')
movie_data = pd.merge(movie_data, keywords,how = "inner", on = 'id')
# movie_data_m
# movie_data

In [28]:
movie_data['cast'] = movie_data['cast'].apply(literal_eval)
movie_data['crew'] = movie_data['crew'].apply(literal_eval)
movie_data['keywords'] = movie_data['keywords'].apply(literal_eval)

In [29]:
# movie_data.cast
# movie_data.crew
def getActor(x):
    if (len(x) >5):
        x = x[:5]
    else:
        x = x
    return [actor['name'].replace(" ","") for actor in x ]

def getDirector(x):
    return [crew['name'].replace(" ", "") for crew in x if crew['job'] == 'Director']

def getKeywords(x):
    return [key['name'] for key in x]


tmdb_movie = movie_data[movie_data['id'].isin(link_small_tmdbId)]
# print(Imdb_movie['cast'].iloc[0])
# print(Imdb_movie['crew'].iloc[1])
# print(tmdb_movie['keywords'].iloc[1])
tmdb_movie['actors'] =tmdb_movie['cast'].apply(getActor)
tmdb_movie['director'] = tmdb_movie['crew'].apply(getDirector)
tmdb_movie['keywords'] = tmdb_movie['keywords'].apply(getKeywords)

In [30]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')
tmdb_movie['keywords'] = tmdb_movie['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

In [52]:
tmdb_movie['details'] = tmdb_movie['keywords'] + tmdb_movie['director'] + tmdb_movie['actors']
# tmdb_movie['details'] = tmdb_movie['details'].apply (lambda x: ' '.join(x))
print(tmdb_movie['details'].iloc[1])
tmdb_movie.to_csv('tmdb_movie.csv')

0       [prison, corrupt, police brut, prison cel, del...
1       [itali, love at first sight, loss of fath, pat...
2       [dc comic, crime fight, secret ident, scarecro...
3       [support group, dual ident, nihil, rage and h,...
4       [transport, brothel, drug deal, boxer, massag,...
5       [vietnam veteran, hippi, mentally dis, run, ba...
6       [factori, concentration camp, hero, holocaust,...
7       [jazz, obsess, conservatori, music teach, new ...
8       [witch, parent child relationship, magic, dark...
9       [rebel, android, asteroid, space battl, snow s...
10      [loss of lov, dream, kidnap, sleep, subconsci,...
11      [itali, riddl, bookshop, self sacrific, mass m...
12      [male friendship, masseus, friendship, aristoc...
13      [saving the world, artificial intellig, father...
14      [italo-american, cuba, vorort, melancholi, pra...
15      [elv, orc, middle-earth (tolkien), based on no...
16      [new york, corrupt, assassin, police brut, los...
17      [indiv

In [48]:
tf_details = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tf_matrix = tf.fit_transform(tmdb_movie['details'])
cosine_sim = linear_kernel(tf_matrix, tf_matrix)
cosine_sim.shape

0       [prison, corrupt, police brut, prison cel, del...
1       [itali, love at first sight, loss of fath, pat...
2       [dc comic, crime fight, secret ident, scarecro...
3       [support group, dual ident, nihil, rage and h,...
4       [transport, brothel, drug deal, boxer, massag,...
5       [vietnam veteran, hippi, mentally dis, run, ba...
6       [factori, concentration camp, hero, holocaust,...
7       [jazz, obsess, conservatori, music teach, new ...
8       [witch, parent child relationship, magic, dark...
9       [rebel, android, asteroid, space battl, snow s...
10      [loss of lov, dream, kidnap, sleep, subconsci,...
11      [itali, riddl, bookshop, self sacrific, mass m...
12      [male friendship, masseus, friendship, aristoc...
13      [saving the world, artificial intellig, father...
14      [italo-american, cuba, vorort, melancholi, pra...
15      [elv, orc, middle-earth (tolkien), based on no...
16      [new york, corrupt, assassin, police brut, los...
17      [indiv

AttributeError: 'list' object has no attribute 'lower'

In [34]:
tmdb_movie = tmdb_movie.reset_index()

In [35]:
titles = tmdb_movie['title']    
indices = pd.Series(data = tmdb_movie.index, index =titles )
indices

title
The Shawshank Redemption                                0
The Godfather                                           1
The Dark Knight                                         2
Fight Club                                              3
Pulp Fiction                                            4
Forrest Gump                                            5
Schindler's List                                        6
Whiplash                                                7
Spirited Away                                           8
The Empire Strikes Back                                 9
Inception                                              10
Life Is Beautiful                                      11
The Intouchables                                       12
Interstellar                                           13
The Godfather: Part II                                 14
The Lord of the Rings: The Return of the King          15
Leon: The Professional                                 16
One Flew

In [ ]:
getRecommendation('The Dark Knight')